In [1]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print(os.getcwd())
os.chdir('/home/jelinek/recetox/')

from keras_preprocessing.image import ImageDataGenerator


from ml.eval import eval_model
from ml.pipeline import FeitDataPipeline
from tensorflow import keras

from tensorflow.keras.callbacks import ReduceLROnPlateau

from cfg import *

from livelossplot import PlotLossesKerasTF


name = "MySimpleCNN-v2-reduced-atrous-convolution"


/home/jelinek/recetox/src/notebooks/FullResolution


In [ ]:
from ml.losses import my_sparse_categorical_loss
from ml.util import FeitClasMapGen
from keras import activations


class MyAtrousCNN(FeitDataPipeline):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.model = self.get_compiled_model()
        self.params.name = name
        self.params.epochs = 200
        self.batch_size = 16
        self.params.tile_size=256
        self.params.number_of_classes = 11

        self.data_loader_training = self.get_data_loader_training()
        self.data_loader_validation = self.get_data_loader_validation()

    @staticmethod
    def convolution_block(block_input, num_filters=256, kernel_size=3, dilation_rate=1, padding="same", use_bias=False):
        x = keras.layers.Conv2D(num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding="same",
            use_bias=use_bias, kernel_initializer=keras.initializers.HeNormal())(block_input)
        """
        Function source: https://keras.io/examples/vision/deeplabv3_plus/
        """
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activations.relu)(x)
        return x

    @staticmethod
    def DilatedSpatialPyramidPooling(dspp_input):
        """
        Function source: https://keras.io/examples/vision/deeplabv3_plus/
        """

        dims = dspp_input.shape
        x = keras.layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
        x = MyAtrousCNN.convolution_block(x, kernel_size=1, use_bias=True)
        out_pool = keras.layers.UpSampling2D(
            size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",)(x)

        out_1 = MyAtrousCNN.convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
        out_6 = MyAtrousCNN.convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
        out_12 = MyAtrousCNN.convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
        out_18 = MyAtrousCNN.convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

        x = keras.layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
        output = MyAtrousCNN.convolution_block(x, kernel_size=1)
        return output


    @staticmethod
    def get_compiled_model():
        inputs = keras.Input(shape=(256, 256, 3))

        x = keras.layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), padding='same')(inputs)

        x = keras.layers.MaxPooling2D(padding='same', pool_size=(2, 2))(x)
        x = keras.layers.BatchNormalization(axis=3, epsilon=1.001e-5)(x)

        x = keras.layers.Conv2D(filters=32, kernel_size=5, strides=(1, 1), padding='same', name="conv128")(x)
        x = keras.layers.MaxPooling2D(padding='same', pool_size=(2, 2))(x)
        x = keras.layers.BatchNormalization(axis=3, epsilon=1.001e-5)(x)

        x = keras.layers.Conv2D(filters=64, kernel_size=5, strides=(1, 1), padding='same', name="conv64")(x)
        x = keras.layers.MaxPooling2D(padding='same', pool_size=(2, 2))(x)
        x = keras.layers.BatchNormalization(axis=3, epsilon=1.001e-5)(x)

        x = keras.layers.Conv2D(filters=128, kernel_size=5, strides=(1, 1), padding='same', name="conv32")(x)
        x = keras.layers.MaxPooling2D(padding='same', pool_size=(2, 2))(x)
        x = keras.layers.BatchNormalization(axis=3, epsilon=1.001e-5)(x)

        x = keras.layers.Conv2D(filters=256, kernel_size=5, strides=(1, 1), padding='same', name="conv16")(x)
        x = keras.layers.MaxPooling2D(padding='same', pool_size=(2, 2))(x)
        x = keras.layers.BatchNormalization(axis=3, epsilon=1.001e-5)(x)

        x = keras.layers.Conv2D(filters=512, kernel_size=5, strides=(1, 1), dilation_rate=2, padding='same', name="conv8")(x)
        x = keras.layers.MaxPooling2D(padding='same', pool_size=(2, 2))(x)
        x = keras.layers.BatchNormalization(axis=3, epsilon=1.001e-5)(x)

        x = keras.layers.Flatten()(x)
        outputs = keras.layers.Dense(units=11, activation='softmax')(x)

        base_model = keras.Model(inputs=inputs, outputs=outputs)

        ## Code below adapted from https://keras.io/examples/vision/deeplabv3_plus/
        x = base_model.get_layer("conv8").output
        x = MyAtrousCNN.DilatedSpatialPyramidPooling(x)

        input_a = keras.layers.UpSampling2D(
            size=(256 // 16 // x.shape[1], 256 // 16 // x.shape[2]),
            interpolation="bilinear",
        )(x)
        input_b = base_model.get_layer("conv16").output
        input_b = MyAtrousCNN.convolution_block(input_b, num_filters=48, kernel_size=1)

        x = keras.layers.Concatenate(axis=-1)([input_a, input_b])
        x = MyAtrousCNN.convolution_block(x)
        x = MyAtrousCNN.convolution_block(x)
        x = keras.layers.MaxPooling2D()(x)
        #x = keras.layers.Down(
        #    size=(256 // x.shape[1], 256 // x.shape[2]),
        #    interpolation="bilinear",
        # )(x)
        model_output = keras.layers.Conv2D(11, kernel_size=(1, 1), padding="same")(x)

        final_model = keras.Model(inputs=inputs, outputs=model_output)
        final_model.summary()

        return final_model

    def execute_pipeline(self, perform_validation=True, save_model=True, perform_test_segmentation=True):
        data_train = self.data_loader_training
        data_valid = self.data_loader_validation

        self.model.compile(loss='sparse_categorical_crossentropy',
                           optimizer=self.get_optimizer(),
                           metrics=['accuracy'])


        self._train_model(data_train, data_valid)

    def get_data_loader_training(self):
        datagen_train = FeitClasMapGen(horizontal_flip=True, vertical_flip=True, samplewise_center=True,
                                           samplewise_std_normalization=True)

        return datagen_train.flow_from_directory(directory=self.params.data_training, color_mode='rgb',
                                                 class_mode='categorical', batch_size=self.params.batch_size,
                                                 shuffle=True,
                                                 target_size=(self.params.tile_size, self.params.tile_size))

    def get_data_loader_validation(self):
        datagen_valid = FeitClasMapGen(samplewise_center=True, samplewise_std_normalization=True)
        return datagen_valid.flow_from_directory(directory=self.params.data_validation, color_mode='rgb',
                                                 class_mode='categorical', batch_size=self.params.batch_size,
                                                 shuffle=True,
                                                 target_size=(self.params.tile_size, self.params.tile_size))

    def get_optimizer(self):
        return keras.optimizers.Adam(learning_rate=1e-3)


    def _train_model(self, data_train, data_valid):

        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                      patience=30, min_lr=1e-4, verbose=1,
                                      cooldown=20)

        class_weights = {k : 1.0 for k in range(self.params.number_of_classes)}
        class_weights[self.params.number_of_classes] = 0.0

        self.model.fit(data_train,
                       steps_per_epoch=250,
                       epochs=20,
                       shuffle=True,
                       validation_data=data_valid,
                       validation_freq=100,
                       verbose=1,
                       validation_steps=10000,
                       callbacks=[self.tensorboard, reduce_lr, PlotLossesKerasTF()])

        print('Ahoj')

In [6]:
pipeline = MyAtrousCNN(train_data_dir='data/Feit_colon-annotation_valid/',
                        valid_data_dir='data/Feit_colon-annotation_valid/')

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_11 (Conv2D)             (None, 256, 256, 16  1216        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_7 (MaxPooling2D)  (None, 128, 128, 16  0          ['conv2d_11[0][0]']              
                                )                                                           

In [8]:
pipeline.execute_pipeline(perform_validation=True, perform_test_segmentation=False)

Epoch 1/20
142/250 [================>.............] - ETA: 9s - loss: nan - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [5]:
pipeline.save_pipeline()

KeyboardInterrupt: 

In [6]:
eval_model(pipeline.model,
           pipeline.get_data_loader_validation(),
           name,
           print_confusion_matrix=True,
           save_misclassified=True)

Initializing training datagen

Processing file 1
Processing grid point 663499 out of 663499
Processing file 2
Processing grid point 1599360 out of 1599360
Processing file 3
Processing grid point 1541550 out of 1541550
Processing file 4


AttributeError: 'FeitClasMapSequence' object has no attribute 'classes'

In [ ]:
pipeline = FeitDataPipeline.load_pipeline(pipeline_name=name)

In [ ]:
from ml.eval import evaluate_segmentation_on_feit_annotation

evaluation_path = Path('data/Feit_colon-annotation_valid/')

segmentation_dir = Path('segmentations') / pipeline.params.name

evaluate_segmentation_on_feit_annotation(evaluation_path, pipeline.build_segmenter(),
                                         32, pipeline.params.class_names,
                                         save_segmentations=True, segmentations_dir=segmentation_dir,
                                         neighbourhood_size=3)